# Data Cleaning
## Step 1: Loading the Dataset

In [1]:
import pandas as pd

file_path = 'data/Online_Retail.xlsx'
original_data = pd.read_excel(file_path)

original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Step 2: Inspecting the dataset

In [2]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [3]:
#Checking for number of missing values
original_data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [4]:
# Output rows with any missing values
missing_rows = original_data[original_data.isnull().any(axis=1)]

# Display the result
print(missing_rows)

       InvoiceNo StockCode                      Description  Quantity  \
622       536414     22139                              NaN        56   
1443      536544     21773  DECORATIVE ROSE BATHROOM BOTTLE         1   
1444      536544     21774  DECORATIVE CATS BATHROOM BOTTLE         2   
1445      536544     21786               POLKADOT RAIN HAT          4   
1446      536544     21787            RAIN PONCHO RETROSPOT         2   
...          ...       ...                              ...       ...   
541536    581498    85099B          JUMBO BAG RED RETROSPOT         5   
541537    581498    85099C   JUMBO  BAG BAROQUE BLACK WHITE         4   
541538    581498     85150    LADIES & GENTLEMEN METAL SIGN         1   
541539    581498     85174                S/4 CACTI CANDLES         1   
541540    581498       DOT                   DOTCOM POSTAGE         1   

               InvoiceDate  UnitPrice  CustomerID         Country  
622    2010-12-01 11:52:00       0.00         NaN  Unit

In [61]:
original_data[original_data['Description'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,GuestCustomer3,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,GuestCustomer6,United Kingdom
1988,536550,85044,NaN,1,2010-12-01 14:34:00,0.0,GuestCustomer7,United Kingdom
2024,536552,20950,NaN,1,2010-12-01 14:34:00,0.0,GuestCustomer8,United Kingdom
2026,536554,84670,NaN,23,2010-12-01 14:35:00,0.0,GuestCustomer10,United Kingdom
...,...,...,...,...,...,...,...,...
280754,561498,21610,NaN,-14,2011-07-27 14:10:00,0.0,GuestCustomer2380,United Kingdom
281615,561555,37477B,NaN,-11,2011-07-28 10:21:00,0.0,GuestCustomer2404,United Kingdom
281616,561557,37477C,NaN,-31,2011-07-28 10:21:00,0.0,GuestCustomer2405,United Kingdom
346849,567207,35592T,NaN,4,2011-09-19 11:01:00,0.0,GuestCustomer2754,United Kingdom


## Step 3: Filling in Missing values

### Filling in Missing values in Description feature

In [8]:
#Using Stock Code to find out the missing values In Decsription 
# Loop through rows with missing Description
for stock_code in original_data[original_data['Description'].isnull()]['StockCode'].unique():
    # Find rows with the same StockCode
    matching_rows = original_data[original_data['StockCode'] == stock_code]
    
    # Check if there are any non-missing Description values
    non_missing_description = matching_rows['Description'].dropna().unique()
    
    # If a non-missing value exists, replace the missing values
    if len(non_missing_description) > 0:
        # Use the first non-missing value (assuming all are the same)
        replacement_value = non_missing_description[0]
        
        # Update missing values in Description for this StockCode
        original_data.loc[
            (original_data['StockCode'] == stock_code) & 
            (original_data['Description'].isnull()), 
            'Description'
        ] = replacement_value

In [9]:
#Renaming Description name that don't look like product name to 'Invalid Description Naming'
original_data.loc[(~original_data['Description'].astype(str).str.isupper()) & (original_data['Description'].astype(str).str.len() <= 10) & (~original_data['Description'].isnull()), 'Description'] = 'Invalid Description Naming'

In [64]:
original_data['Description'] = original_data['Description'].fillna('No description')

In [10]:
"""
array(['Discount', 'Manual', 'Amazon', '?', 'Check', 'Damages',
       'Faulty', 'Found', 'Counted', 'Given Away', 'Dotcom', 'Showroom',
       'Adjustment', 'Dotcom Set', 'Broken', 'Throw Away', '?Lost',
       'Damages?', 'Cracked', 'Damaged', 'Display', 'Missing', 'Returned',
       'Wrong Code', 'Adjust', 'Crushed', 'Samples', 'Mailout ',
       'Mailout', 'Wet/Rusty', 'Smashed', 'Ebay', '?Display?',
       'Sold As 1', '?Missing', 'Test', '??', 'Found Box', 'Dagamed',
       "Can'T Find", 'Mouldy', 'Sale Error', 'Breakages', 'Missing?',
       'Wet Rusty', '???Lost', 'John Lewis', 'Check?', '?? Missing',
       'Wet Pallet', '???Missing', 'Wet?', 'Lost??', '???', 'Wet',
       'Wet Boxes', 'Mixed Up', 'Lost'], dtype=object)
"""

'\narray([\'Discount\', nan, \'Manual\', \'Amazon\', \'?\', \'Check\', \'Damages\',\n       \'Faulty\', \'Found\', \'Counted\', \'Given Away\', \'Dotcom\', \'Showroom\',\n       \'Adjustment\', \'Dotcom Set\', \'Broken\', \'Throw Away\', \'?Lost\',\n       \'Damages?\', \'Cracked\', \'Damaged\', \'Display\', \'Missing\', \'Returned\',\n       \'Wrong Code\', \'Adjust\', \'Crushed\', \'Samples\', \'Mailout \',\n       \'Mailout\', \'Wet/Rusty\', \'Smashed\', \'Ebay\', \'?Display?\',\n       \'Sold As 1\', \'?Missing\', \'Test\', \'??\', \'Found Box\', \'Dagamed\',\n       "Can\'T Find", \'Mouldy\', \'Sale Error\', \'Breakages\', \'Missing?\',\n       \'Wet Rusty\', \'???Lost\', \'John Lewis\', \'Check?\', \'?? Missing\',\n       \'Wet Pallet\', \'???Missing\', \'Wet?\', \'Lost??\', \'???\', \'Wet\',\n       \'Wet Boxes\', \'Mixed Up\', \'Lost\'], dtype=object)\n'

### Filling Missing values in CustomerID feature

In [42]:
#Checking if the null values in 'CustomerID' can be gotten from the 'InvoiceNo'
missing_customerid = original_data[original_data['CustomerID'].isnull()]
missing_invoice_counts = missing_customerid['InvoiceNo'].value_counts()
mic = missing_invoice_counts.index.to_list()

non_missing_customerid = original_data[~original_data['CustomerID'].isnull()]
non_missing_invoice_counts = non_missing_customerid['InvoiceNo'].value_counts()
nmic = non_missing_invoice_counts.index.to_list()

for invoice_counts in mic:
    if invoice_counts in nmic:
        print(i)
        

In [52]:
#Assign GuestCustomers unique value to each CustomerID null value with unique InvoicNo
nan_invoices = original_data[original_data['CustomerID'].isnull()]['InvoiceNo'].unique()

guest_mapping = {invoice: f"GuestCustomer{idx + 1}" for idx, invoice in enumerate(nan_invoices)}
guest_mapping

original_data['CustomerID'] = original_data.apply(
    lambda row: guest_mapping[row['InvoiceNo']] if pd.isna(row['CustomerID']) else row['CustomerID'],
    axis=1
)

## Step 4: Dropping Duplicates

In [70]:
original_data = original_data.drop_duplicates()

## Step 5: Save Cleaned Data

In [72]:
original_data.to_csv('data/cleaned_sales_data.csv', index=False)